In [244]:
import config
import json
import os
import random
random.seed(1234)
import numpy as np; np.random.seed(1234)
from utils_picking import my_read_old, intersect, round_percent, get_indices_diff_list_suffix, vqa_score_list, ch_atleast_once, worst_case_acc
import seaborn as sns; sns.set()

In [ ]:
ques_type_dir = 0
TEST_SPLIT = 'val2014'
DIST_NORM = 1
MODEL = 'snmn'

orig_90_10_vs_edit_90_10 = 1  ##### which set splits for orig you want to test
snmn_lr_25_e_4 = 0  # make sure snmn_lr_1e3_cvpr_check lr =0 
snmn_lr_1e3_cvpr_check = 0  
ques_type = MINI_DIR

In [247]:
## orig_all vs edit_10
edit_root_dir_qa = config.iv_q  

coco_json = 'coco_cat_ids.json'

#### questions keys: 'image_id', 'question', 'question_id'
standard_questions_val_json = '/BS/databases10/VQA_v2/Questions/v2_OpenEnded_mscoco_'  + TEST_SPLIT + '_questions.json'
res_file_q = 'v2_OpenEnded_mscoco_' + TEST_SPLIT + '_questions.pickle'
standard_questions_edit_val_json = os.path.join(edit_root_dir_qa, res_file_q)


## ann keys: 'image_id', 'question_id', 'answers' , 'multiple_choice_answer'(the most frequent answer), 'question_type', 'answer_type'
standard_annotations_val_json = '/BS/databases10/VQA_v2/Annotations/v2_mscoco_'  + TEST_SPLIT + '_annotations.json'
res_file_a = 'v2_mscoco_' + TEST_SPLIT + '_annotations.json'
standard_annotations_edit_val_json = os.path.join(edit_root_dir_qa, res_file_a)

if orig_90_10_vs_edit_90_10:
    orig_root_dir_qa = '/BS/vedika2/nobackup/thesis/mini_datasets_qa_CNN_finetune_testing/' + ques_type + '/orig_90_10/'
    edit_root_dir_qa = '/BS/vedika2/nobackup/thesis/mini_datasets_qa_CNN_finetune_testing/' + ques_type + '/edit_10/'
    coco_json = '/BS/vedika2/nobackup/snmn/coco_cat_ids.json'

    #### questions keys: 'image_id', 'question', 'question_id'
    res_file_q = 'v2_OpenEnded_mscoco_' + TEST_SPLIT + '_questions.json'
    standard_questions_val_json = os.path.join(orig_root_dir_qa, res_file_q)
    standard_questions_edit_val_json = os.path.join(edit_root_dir_qa, res_file_q)


    ## ann keys: 'image_id', 'question_id', 'answers' , 'multiple_choice_answer'(the most frequent answer), 'question_type', 'answer_type'
    res_file_a = 'v2_mscoco_' + TEST_SPLIT + '_annotations.json'
    standard_annotations_val_json = os.path.join(orig_root_dir_qa, res_file_a)
    standard_annotations_edit_val_json = os.path.join(edit_root_dir_qa, res_file_a)

In [248]:
### make changes here

if MODEL == 'snmn':
    results_edit_val_old = '/BS/vedika2/nobackup/snmn/exp_vqa/eval_outputs_vqa_v2/vqa_v2_scratch_train/vqa_v2_edited_val2014_vqa_v2_scratch_train_15000_results.pickle'
    results_val_old = '/BS/vedika2/nobackup/snmn/exp_vqa/eval_outputs_vqa_v2/vqa_v2_scratch_train/vqa_v2_val2014_vqa_v2_scratch_train_15000_results.pickle'
    ## standard_vocab file_model_specific
    standard_vocab_ans_file = '/BS/vedika2/nobackup/snmn/exp_vqa/data/answers_vqa.txt'
    with open(standard_vocab_ans_file) as f:
        ans_vocab_list = f.read().splitlines()
    
    if snmn_lr_25_e_4:
        results_edit_val_old = '/BS/vedika2/nobackup/snmn/exp_vqa/eval_outputs_vqa_v2/vqa_v2_scratch_train_25lr/vqa_v2_edited_val2014_vqa_v2_scratch_train_25lr_30_results.pickle'
        results_val_old = '/BS/vedika2/nobackup/snmn/exp_vqa/eval_outputs_vqa_v2/vqa_v2_scratch_train_25lr/vqa_v2_val2014_vqa_v2_scratch_train_25lr_30_results.pickle'

    if snmn_lr_1e3_cvpr_check:        
        results_val_old =  '/BS/vedika2/nobackup/snmn/exp_vqa/eval_outputs_vqa_v2/vqa_v2_scratch_train_cvpr_check/vqa_v2_val2014_vqa_v2_scratch_train_cvpr_check_15_results.pickle'
        results_edit_val_old =   '/BS/vedika2/nobackup/snmn/exp_vqa/eval_outputs_vqa_v2/vqa_v2_scratch_train_cvpr_check/vqa_v2_edited_val2014_vqa_v2_scratch_train_cvpr_check_15_results.pickle'
        
elif MODEL == 'SAAA':
    results_edit_val_old ='/BS/vedika3/nobackup/pytorch-vqa/logs/edit_val_with_attn_.pickle'
    results_val_old ='/BS/vedika3/nobackup/pytorch-vqa/logs/val_with_attn_.pickle'
    ## standard_vocab file_model_specific
    vocab = '/BS/vedika3/nobackup/pytorch-vqa/vocab.json'
    with open(vocab, 'r') as f:
        ans_vocab = json.load(f)["answer"]
        ans_vocab_list = [k for k, v in ans_vocab.items()]
        #{v: k for k, v in ans_vocab.items()}   ### is a dictionary here but will work: keys- index- 0,1,2...

elif MODEL == 'CNN_LSTM':
    results_edit_val_old = '/BS/vedika3/nobackup/pytorch-vqa/logs/edit_val_no_attn_.pickle'
    results_val_old = '/BS/vedika3/nobackup/pytorch-vqa/logs/val_no_attn_.pickle'
    ## standard_vocab file_model_specific
    vocab = '/BS/vedika3/nobackup/pytorch-vqa/vocab.json'
    with open(vocab, 'r') as f:
        ans_vocab = json.load(f)["answer"]
        ans_vocab_list = [k for k, v in ans_vocab.items()]
        # {v: k for k, v in ans_vocab.items()}   ### is a dictionary here but will work: keys- index- 0,1,2...
    dir_prefix = 'CNN_LSTM'
    
img_dir = '/BS/databases10/VQA_v2/Images/' + TEST_SPLIT + '/'
edit_img_dir = '/BS/vedika2/work/thesis/final_edited_VQA_v2/Images/' + TEST_SPLIT + '/'
img_prefix_name = 'COCO_' + TEST_SPLIT + '_'



coco_json = '/BS/vedika2/nobackup/snmn/coco_cat_ids.json'
with open(coco_json) as file:
    coco_f = json.load(file)
coco_dict = {}
for idx, details in enumerate(coco_f):
    # print(details)
    coco_dict[details['id']] = details['name']
coco_dict_inv = {}
for idx, details in enumerate(coco_f):
    # print(details)
    coco_dict_inv[details['name']] = details['id']

ques_type_file = '/BS/vedika3/nobackup/VQA_helper_tools_official/QuestionTypes/mscoco_question_types.txt'
with open(ques_type_file) as f:
    ques_type_off = f.read().splitlines()
    
ans_type_off = ['yes/no', 'number', 'other']


In [252]:
qid_val, pred_ans_val , ss_vc_val, img_ids_val, ques_val, all_ans_val, ques_type_val, ans_type_val = my_read_old(results_val_old, standard_questions_val_json, standard_annotations_val_json)

6.4199817180633545


In [253]:
qid_edit_val, pred_ans_edit_val , ss_vc_edit_val, img_ids_edit_val, ques_edit_val, all_ans_edit_val, ques_type_edit_val, ans_type_edit_val = my_read_old(results_edit_val_old, standard_questions_edit_val_json, standard_annotations_edit_val_json)

10.360442876815796


In [254]:
if orig_90_10_vs_edit_90_10:
    masking_indices_where_qid_edit_but_no_orig_qid = [idx for idx, i in enumerate(qid_edit_val) if i not in qid_val]
    stop_idx = masking_indices_where_qid_edit_but_no_orig_qid[0]
    qid_edit_val = qid_edit_val[0:stop_idx]
    qid_edit_val, pred_ans_edit_val, ss_vc_edit_val, img_ids_edit_val, ques_edit_val, all_ans_edit_val, ques_type_edit_val, ans_type_edit_val = [
        i[0:stop_idx] for i in
        [qid_edit_val, pred_ans_edit_val, ss_vc_edit_val, img_ids_edit_val, ques_edit_val, all_ans_edit_val,
         ques_type_edit_val, ans_type_edit_val]]
    print(stop_idx)


108239


In [256]:
len_val = len(img_ids_val)
len_edit_val = len(img_ids_edit_val)
all_indices_val = np.arange(len_val)
all_indices_edit_val = np.arange(len_edit_val)

print('size of original set:', len_val)
print('size of edited set:', len_edit_val)
print('no of unique images in val_set: ',len(set(img_ids_val)))
print('no of unique images in edit_val_set: ',len(set(img_ids_edit_val)))
print('#unique questions in original set: ',len(list(set(qid_val))) )
print('#unique questions in edited set: ', len(list(set(qid_edit_val))))
    
if len(list(set(qid_val))) != len(list(set(qid_edit_val))) :
    print('not every question in orig_val made it to edit_val=> for ', len(list(set(qid_val)))- len(list(set(qid_edit_val)))  ,' questions- no legit edited IQA possible')

qid_gt_ans_label = {}
for idx,a in enumerate(qid_val):
    qid_gt_ans_label[a] = all_ans_val[idx]
    
qid_predans_val = {}
for idx, a in enumerate(qid_val):
    qid_predans_val.setdefault(a, []).append(pred_ans_val[idx])
    
qid_predans_idx_val = {}
for idx, a in enumerate(qid_val):
    qid_predans_idx_val.setdefault(a, []).append(idx)
    
#idx in case here refers to len(val and edit_val- order hai - so relax)    
qid_predans_edit_val = {}
for idx, a in enumerate(qid_edit_val):
    qid_predans_edit_val.setdefault(a, []).append(pred_ans_edit_val[idx])
    #qid_predans_edit_val[a] = (pred_ans_edit_val[idx])   
    
qid_predans_idx_edit_val = {}
for idx, a in enumerate(qid_edit_val):
    qid_predans_idx_edit_val.setdefault(a, []).append(idx)    
    
# qid_predans_imgid_edit_val = {}
# for idx, a in enumerate(qid_edit_val):
#     qid_predans_imgid_edit_val.setdefault(a, []).append(img_ids_edit_val[idx])   

## creating dictionary for val set - to facilitate extensions based on q_id index
qid_ss_predans_val = {}
for idx, a in enumerate(qid_val):
    qid_ss_predans_val[a] = (ss_vc_val[idx], pred_ans_val[idx], qid_val[idx],idx, all_ans_val[idx])

extended_ss_vc_val = [qid_ss_predans_val[q_id][0] for q_id in qid_edit_val]
extended_pred_ans_val = [qid_ss_predans_val[q_id][1] for q_id in qid_edit_val] 
#extended_qid_val = [qid_ss_predans_val[q_id][2] for q_id in qid_edit_val]

collapsed_pred_ans_val = [qid_ss_predans_val[q_id][1] for q_id in set(qid_edit_val)] 
collapsed_all_ans_val = [qid_ss_predans_val[q_id][4] for q_id in set(qid_edit_val)] 
collapsed_indices = [qid_ss_predans_val[q_id][3] for q_id in set(qid_edit_val)] 




size of original set: 63219
size of edited set: 108239
no of unique images in val_set:  29376
no of unique images in edit_val_set:  53855
#unique questions in original set:  63219
#unique questions in edited set:  45076
not every question in orig_val made it to edit_val=> for  18143  questions- no legit edited IQA possible


In [258]:
###                                         STATISTICS
assert len(collapsed_indices) == len(set(qid_edit_val))

ind_where_10_ans_same = [idx for idx, i in enumerate(all_ans_val) if len(set(i))==1 ]  

## one match answer is good
accuracy_ind_before = [i for i in range(len_val) if ans_vocab_list[pred_ans_val[i]] in all_ans_val[i]]

accuracy_ind_bf_10_ans_same = [i for i in ind_where_10_ans_same if ans_vocab_list[pred_ans_val[i]] in all_ans_val[i]]

accuracy_ind_before_extended = [i for i in range(len_edit_val) if ans_vocab_list[extended_pred_ans_val[i]] in all_ans_edit_val[i]]
accuracy_ind_after = [i for i in range(len_edit_val) if ans_vocab_list[pred_ans_edit_val[i]] in all_ans_edit_val[i]]
print('accuracy before editing, one answer match is good', 
      round_percent(len(accuracy_ind_before) / len_val), len(accuracy_ind_before))

print('accuracy before editing_10_ans_same, one answer match is good', 
      round_percent(len(accuracy_ind_bf_10_ans_same) / len(ind_where_10_ans_same)), len(ind_where_10_ans_same))


print('accuracy before editing_extended', 
      round_percent(len(accuracy_ind_before_extended) / len_edit_val), len(accuracy_ind_before_extended))
print('accuracy after editing, one answer match is good', 
      round_percent(len(accuracy_ind_after) / len_edit_val), len(accuracy_ind_after))

accuracy_ind_before_collapsed = [i for i in range(len(set(qid_edit_val))) if ans_vocab_list[collapsed_pred_ans_val[i]] in collapsed_all_ans_val[i]]
print('accuracy before editing_collapsed', 
      round_percent(len(accuracy_ind_before_collapsed) / len(set(qid_edit_val))))

# accuracy_ind_before_c = [i for i in collapsed_indices if ans_vocab_list[pred_ans_val[i]] in all_ans_val[i]]
# print('accuracy before editing, one answer match is good', 
#       round_percent(len(accuracy_ind_before_c) /len(set(qid_edit_val))))

worst_case_acc(qid_predans_edit_val.keys(),qid_predans_idx_edit_val, \
               qid_predans_edit_val,ans_vocab_list, qid_gt_ans_label, if_print=True)
#worst_case_acc(qid_predans_val.keys(), qid_predans_val,ans_vocab_list, qid_gt_ans_label)
#ch_atleast_once(qid_predans_edit_val.keys(), qid_predans_edit_val, qid_predans_val ,ans_vocab_list)


##official_way
off_score_val = vqa_score_list(all_ans_val, pred_ans_val, ans_vocab_list)
off_score_val_extended = vqa_score_list(all_ans_edit_val, extended_pred_ans_val, ans_vocab_list)
off_score_val_collapsed = vqa_score_list(collapsed_all_ans_val, collapsed_pred_ans_val, ans_vocab_list)
off_score_edit_val = vqa_score_list(all_ans_edit_val, pred_ans_edit_val, ans_vocab_list)
print('official accuracy before editing', round_percent(np.sum(off_score_val) / len_val))
print('official accuracy before editing_all_10_ans_same', round_percent(len(accuracy_ind_bf_10_ans_same) / len(ind_where_10_ans_same)), len(ind_where_10_ans_same))



print('official accuracy before editing_extended', round_percent(np.sum(off_score_val_extended) / len_edit_val))
print('official accuracy after editing', round_percent(np.sum(off_score_edit_val) / len_edit_val))
print('official accuracy before editing_collapsed', round_percent(np.sum(off_score_val_collapsed) / len(collapsed_indices)))

# ##SNMN- authors way
# author_val_acc = np.sum([pred_ans_val[i] == gt_ans_used_val[i] for i in range(len_val)]) / len_val
# author_val_extended_acc = np.sum([extended_pred_ans_val[i] == gt_ans_used_edit_val[i] for i in range(len_edit_val)]) / len_edit_val
# author_edit_val_acc = np.sum([pred_ans_edit_val[i] == gt_ans_used_edit_val[i] for i in range(len_edit_val)]) / len_edit_val
# print('authors accuracy before editing', round_percent(author_val_acc))
# print('authors accuracy before editing_extended', round_percent(author_val_extended_acc))
# print('authors accuracy after editing', round_percent(author_edit_val_acc))

accuracy before editing, one answer match is good 66.039 41749
accuracy before editing_10_ans_same, one answer match is good 66.039 63219
accuracy before editing_extended 65.447 70839
accuracy after editing, one answer match is good 65.146 70513
accuracy before editing_collapsed 65.671
worst case accuracy (45076):  63.178
best case accuracy (45076):  67.517
worst case official accuracy (45076):  63.178
best case official accuracy (45076):  67.517
official accuracy before editing 66.039
official accuracy before editing_all_10_ans_same 66.039 63219
official accuracy before editing_extended 65.447
official accuracy after editing 65.146
official accuracy before editing_collapsed 65.671


In [259]:
### for entire set counting how mnay labels flipped
labels_flipped_count = np.sum([extended_pred_ans_val[i] != val for i, val in enumerate(pred_ans_edit_val)])
labels_remained_same_count = np.sum([extended_pred_ans_val[i] == val for i, val in enumerate(pred_ans_edit_val)])
assert (labels_flipped_count + labels_remained_same_count == len(pred_ans_edit_val))
print('#labels flipped for entire set=', labels_flipped_count,'      ' ,
      round_percent(labels_flipped_count / len_edit_val), '%')
      
print('#labels remained same=', labels_remained_same_count,'       ', 
      round_percent(labels_remained_same_count / len_edit_val), '%')

lab_fl_ind = [i for i in range(len_edit_val) if extended_pred_ans_val[i] != pred_ans_edit_val[i]]
# label i.e ans was wrong before- right now- one match to 10gt ans is okay
lab_fl_pos = [i for i in lab_fl_ind if ans_vocab_list[extended_pred_ans_val[i]] not in all_ans_edit_val[i] and ans_vocab_list[pred_ans_edit_val[i]] in all_ans_edit_val[i]]
# label i.e ans was right before- now wrong - one match to 10gt ans is okay
lab_fl_neg = [i for i in lab_fl_ind if ans_vocab_list[extended_pred_ans_val[i]] in all_ans_edit_val[i] and ans_vocab_list[pred_ans_edit_val[i]] not in all_ans_edit_val[i]]
lab_fl_right = [i for i in lab_fl_ind if ans_vocab_list[extended_pred_ans_val[i]] in all_ans_edit_val[i] and ans_vocab_list[pred_ans_edit_val[i]] in all_ans_edit_val[i]]
lab_fl_wrong = [i for i in lab_fl_ind if ans_vocab_list[extended_pred_ans_val[i]] not in all_ans_edit_val[i] and ans_vocab_list[pred_ans_edit_val[i]] not in all_ans_edit_val[i]]
print('#labels flipped that show a positive change', len(lab_fl_pos), '         ', 
      round_percent(len(lab_fl_pos) / len_edit_val), '%' )
print('#labels flipped that show a negative change', len(lab_fl_neg), '         ', 
      round_percent(len(lab_fl_neg) / len_edit_val), '%')
print('#labels flipped that are both right', len(lab_fl_right), '         ', round_percent(len(lab_fl_right) / len_edit_val), '%')
print('#labels flipped that are both wrong', len(lab_fl_wrong), '         ', round_percent(len(lab_fl_wrong) / len_edit_val), '%')
assert(len(lab_fl_pos) + len(lab_fl_neg) + len(lab_fl_right) + len(lab_fl_wrong) == len(lab_fl_ind))


lab_sm_ind = [i for i in range(len_edit_val) if extended_pred_ans_val[i] == pred_ans_edit_val[i]]
# labels that remained exactly same- so two cases possible- either right/wrong
lab_sm_right = [i for i in lab_sm_ind if ans_vocab_list[pred_ans_edit_val[i]] in all_ans_edit_val[i]]
lab_sm_wrong = [i for i in lab_sm_ind if ans_vocab_list[pred_ans_edit_val[i]] not in all_ans_edit_val[i]]
print('#labels same that are both right', len(lab_sm_right), '         ', round_percent(len(lab_sm_right)/len_edit_val), '%')
print('#labels same that are both wrong', len(lab_sm_wrong), '         ', round_percent(len(lab_sm_wrong) / len_edit_val), '%')
assert (len(lab_sm_right) + len(lab_sm_wrong) == len(lab_sm_ind))

#labels flipped for entire set= 7059        6.522 %
#labels remained same= 101180         93.478 %
#labels flipped that show a positive change 2758           2.548 %
#labels flipped that show a negative change 3084           2.849 %
#labels flipped that are both right 0           0.0 %
#labels flipped that are both wrong 1217           1.124 %
#labels same that are both right 67755           62.598 %
#labels same that are both wrong 33425           30.881 %
